<a href="https://colab.research.google.com/github/heegeunyoon/python101/blob/main/1_4_data_analysis_from_ecos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECOS open API
- https://ecos.bok.or.kr/api/#/
- 회원가입 후, '서비스 이용 - 인증키 신청'
    - 아래 url에서 your_auth_key 부분을 인증키로 대체
- '개발 가이드 - 언어별 개발 가이드 - 파이썬' 참고
- '개발 가이드 - 통계코드검색'에서 원하는 통계항목의 코드 검색

'1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)'

- 인증키를 이용하여 2010년부터 2020년까지, 10,000개 이내 자료를 국문 json 파일 형식으로 입수하기 위한 url 주소 생성

ecos open api웹페이지에서 인증키 복사

In [ ]:
your_auth_key = "xxxxxxxxxxxx"

In [ ]:
url = f'http://ecos.bok.or.kr/api/StatisticSearch/{your_auth_key}/json/kr/1/10000/102Y004/A/1950/2023/ABA1/?/?/'
print(url)

In [ ]:
import json
from urllib.request import urlopen

result = urlopen(url)
data = json.loads(result.read()) # 지정된 URL에서 JSON 형식의 데이터를 가져와서 Python의 데이터 구조로 변환하는 과정(파싱)

In [ ]:
type(data)

In [ ]:
data.keys() # 딕셔너리의 키값

In [ ]:
data = data['StatisticSearch']

In [ ]:
data.keys()

In [ ]:
data['list_total_count']

In [ ]:
data['row']

In [ ]:
data = data['row']

In [ ]:
data

딕셔너리 데이터를 이용하여 판다스 데이터프레임 생성

In [ ]:
import pandas as pd

df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df = df.set_index('TIME')
df = df['DATA_VALUE'] # df가 시리즈로 추출

In [ ]:
type(df)

In [ ]:
df.index = pd.to_datetime(df.index)
df.index.names = ['Dates']
df = df.to_frame('DATA_VALUE')
df

In [ ]:
type(df)

In [ ]:
df.index = pd.to_datetime(df.index)
df

In [ ]:
from pandas.tseries.offsets import YearEnd, QuarterEnd, MonthEnd, YearBegin, QuarterBegin, MonthBegin
import json
from urllib.request import urlopen
import pandas as pd


def necos(code = '102Y004', item1 = 'ABA1', item2 = '?', item3 = '?',
         freq = 'A', first = None, last = None, N = '10000',
         detail = True, col = None):

    if first is None:
        if (freq == 'M'):
            first = '190001'
            last = '210012'
        elif (freq == 'Q'):
            first = '1900Q1'
            last = '2100Q4'
        elif (freq == 'A'):
            first = '1900'
            last = '2100'
        elif (freq == 'D'):
            first = '20200101'
            last = '20301231'
            print('Data is being retreived from 2020-01-01.')
        else:
            print('freq must be one of D, M, Q, A.')
            return

    '''retreive monthly, quarterly, annul time series from ecos.
    run 'open_ecosapi() to explore ecos api codes.'''
    ecos_key = f"http://ecos.bok.or.kr/api/StatisticSearch/{your_auth_key}"

    url  = f"{ecos_key}/json/kr/1/{N}/{code}/{freq}/{first}/{last}/{item1}/{item2}/{item3}/"
    result = urlopen(url)
    data = json.loads(result.read())
    data = data["StatisticSearch"]["row"]
    df = pd.DataFrame(data)

    if detail: # 데이터 정보 프린트
        print(f"통계: {df.loc[0, 'STAT_NAME']}",
            f"단위: {df.loc[0, 'UNIT_NAME']}",
            f"기간: {df.loc[0, 'TIME']} - {df.loc[df.index[-1], 'TIME']}",
            f"항목: {df.loc[0, 'ITEM_NAME1']}",)

    df = df.set_index("TIME")
    df.index.names = ['DATE']

    if (freq == 'M'):
        df.index = pd.to_datetime(df.index, format='%Y%m')
    elif (freq == 'Q'):
        df.index = pd.to_datetime(df.index)
    elif (freq == 'A'):
        df.index = pd.to_datetime(df.index, format='%Y')
    elif (freq == 'D'):
        df.index = pd.to_datetime(df.index, format='%Y%m%d')

    df["DATA_VALUE"] = df["DATA_VALUE"].astype("float")

    return df['DATA_VALUE'].to_frame(col)

In [ ]:
cash = necos('102Y001', 'ABA203', freq='M', col = 'cash')

In [ ]:
cash

In [ ]:
kospi = necos('802Y001', '0001000', freq='D', col = '코스피')

In [ ]:
kospi

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

!apt-get -qq install fonts-nanum


fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설정

plt.rcParams['axes.unicode_minus'] = False # 폰트가 한글인 경우, - 표시 오류 수정

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 5))

kospi.plot(ax=ax[0])
kospi.pct_change(365).plot(ax=ax[1])

